# Generación de Texto usando una Recurrent Neuronal Network del tipo RNN básica, LSTM o GRU
Basado en https://www.tensorflow.org/tutorials/text/text_generation

1) Cargar las librerías:

In [1]:
#@title Librerías a usar
import tensorflow as tf
import numpy as np
import os
import csv

print("Librerías cargadas")

Librerías cargadas


In [2]:
#@title Define clases auxiliares


# define la clase para el modelo
class RNNCustomModel(tf.keras.Model):
  def __init__(self, capa_oculta_tipo, vocab_size, embedding_dim, rnn_units):
    super().__init__(self, name="GeneradorTexto")
    # datos de config
    self.tipoModelo = capa_oculta_tipo
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.rnn_units = rnn_units
    # capa de entrada
    if (self.tipoModelo == 'LSTM') or (self.tipoModelo == 'GRU'): 
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="entrada")
    else:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[1, None], name="entrada")
    # capa oculta    
    if self.tipoModelo == 'LSTM': 
      self.hdd = tf.keras.layers.LSTM(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    name="oculta")

    elif self.tipoModelo == 'GRU': 
      self.hdd = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    name="oculta")
    else:
        self.hdd = tf.keras.layers.SimpleRNN(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform',
                      name="oculta")
    # capa de salida
    self.dense = tf.keras.layers.Dense(vocab_size, name="salida")

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.hdd.get_initial_state(x)
    if self.tipoModelo == 'GRU': 
      x, states = self.hdd(x, initial_state=states, training=training)
    else:
      states = self.hdd(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

# clases para generar texto
class GeneradorTexto:

  def __init__(self, model=None, char2idx=None, idx2char=None, caracterJoin=None):
    self.model = model
    self.char2idx = char2idx
    self.idx2char = idx2char
    self.caracterJoin = caracterJoin

  # define función auxiliar para devolver predicción de texto
  def generar(self, temperature=0.1, texto_inicial=' ', cant_generar=100):

    # Converting our start string to numbers (vectorizing)
    if self.caracterJoin == '':
      aux_input = texto_inicial
    else:
      aux_input = texto_inicial.split(self.caracterJoin)
    input_eval = [self.char2idx[s] for s in aux_input]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    self.model.reset_states()
    for i in range(cant_generar):
        predictions = self.model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(self.idx2char[predicted_id])

    return (texto_inicial + self.caracterJoin.join(text_generated))

  # define función para grabar el modelo con toda la información asociada
  def grabar(self, dir):    
    print("\n")
    # crea el directorio
    if not os.path.exists(dir):
      os.mkdir(dir)
    # exporta los pesos
    pesosAr = dir + '/pesos'
    self.model.save_weights(pesosAr, save_format='tf')
    print("Pesos del modelo grabados en ", dir)    
    datosAr = dir + '/configModelo.csv'
    # exporta los datos
    with open(datosAr, mode='w') as csvfile:
      wr = csv.writer(csvfile, delimiter=',')
      # para model
      wr.writerow([self.model.tipoModelo])
      wr.writerow([self.model.vocab_size])
      wr.writerow([self.model.embedding_dim])
      wr.writerow([self.model.rnn_units])
      # para generar texto
      wr.writerow(self.idx2char) 
      wr.writerow(self.caracterJoin)
    print('Datos asociados al modelo grabados en' + datosAr)
    print("\n")
    return self

  # define función para cargar un modelo con toda la información asociada
  def cargar(self, dir):
    print("\n")
    # controla que el directorio exista
    if not os.path.exists(dir):
      print("No existe el directorio a cargar!")
      return None
    # carga datos de configuración
    datosAr = dir + '/configModelo.csv'
    with open(datosAr, mode='r') as csvfile:
      r = csv.reader(csvfile, delimiter=',')
      # para model
      capa_oculta_tipo = r.__next__()[0]
      vocab_size = int(r.__next__()[0])
      embedding_dim = int(r.__next__()[0])
      rnn_units = int(r.__next__()[0])
      # para generar texto
      self.idx2char = r.__next__()
      self.char2idx = {u:i for i, u in enumerate(self.idx2char)}
      auxCaracterJoin = r.__next__()
      if len(auxCaracterJoin)==0:
        self.caracterJoin = ''
      else:
        self.caracterJoin = ' '
    print('Datos asociados al modelo cargados de' + datosAr)
    # crea el modelo y carga los pesos
    # crea el modelo
    self.model = RNNCustomModel(
        capa_oculta_tipo = capa_oculta_tipo,
        vocab_size=vocab_size,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units)
    pesosAr = dir + '/pesos'
    self.model.load_weights(pesosAr)
    print("Pesos del modelo cargados de ", dir)    
    print("\n")
    return self

print("Clases auxiliares definidas")

Clases auxiliares definidas


2) Cargar el texto base a procesar:

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demoML/texto/'  #@param {type:"string"}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
nombre_archivo = "Cantata_del_adelantado_Don_Rodrigo_Diaz_de_Carreras.txt"  #@param {type:"string"}

# levanta el archivo de texto del Drive para procesar
text_cargado = open("".join([path, nombre_archivo]), 'rb').read().decode(encoding='utf-8', errors='ignore')

print("> Archivo cargado:")
print ('\n -- Tamaño total del texto: {} caracteres'.format(len(text_cargado)))

# muestra los primeros 250 caracteres del texto
print("\n -- Ejemplo: \n", text_cargado[:250])

> Archivo cargado:

 -- Tamaño total del texto: 20630 caracteres

 -- Ejemplo: 
 ﻿@SONG: Cantata del adelantado Don Rodrigo Diaz de Carreras...

[INTRODUCCIÓN]

Mastropiero era un apasionado de la investigación histórica.

Se pasaba largas horas en la biblioteca de la opulenta marquesa de
Quintanilla, cuyos volúmenes le ap


3) Preparar el texto base a procesar:

In [8]:
#@title Limpiar el texto

sacar_caracteres_especiales = True #@param {type:"boolean"}
sacar_signos_puntuacion = True #@param {type:"boolean"}
sacar_otros_signos = True #@param {type:"boolean"}
sacar_acentos = True #@param {type:"boolean"}
pasar_minusculas = True #@param {type:"boolean"}

# hace una copia por si se vuelve a ejecutar
text = str(text_cargado)

# siempre saca símbolo de inicio
text = text.replace('\ufeff', ' ')

if sacar_caracteres_especiales:
  text = text.replace('\n', ' ')
  text = text.replace('\t', ' ')
  text = text.replace('\r', ' ')   

if sacar_signos_puntuacion:
  text = text.replace(',', ' ')
  text = text.replace(';', ' ')
  text = text.replace('.', ' ')
  text = text.replace('¡', ' ')
  text = text.replace('¿', ' ')
  text = text.replace('!', ' ')
  text = text.replace('?', ' ')  

if sacar_otros_signos:
  text = text.replace('-', ' ')
  text = text.replace(':', ' ')
  text = text.replace('\'', ' ')
  text = text.replace('"', ' ')
  text = text.replace('“', ' ')
  text = text.replace('”', ' ')
  text = text.replace('`', ' ')
  text = text.replace('[', ' ')
  text = text.replace(']', ' ')
  text = text.replace('(', ' ')
  text = text.replace(')', ' ')
  text = text.replace('<', ' ')
  text = text.replace('>', ' ')
  text = text.replace('=', ' ')
  text = text.replace('/', ' ')
  text = text.replace('@', ' ')
  text = text.replace('~', ' ')
  text = text.replace('*', ' ')
  text = text.replace('_', ' ')

# pasa todo a minúsculas
if pasar_minusculas:
  text = text.lower()

# eliminar acentos (reemplaza por letra sin acento)
if sacar_acentos:
  text = text.replace('á', 'a')
  text = text.replace('é', 'e')
  text = text.replace('í', 'i')
  text = text.replace('ó', 'o')
  text = text.replace('ú', 'u')
  text = text.replace('Á', 'a')
  text = text.replace('É', 'e')
  text = text.replace('Í', 'i')
  text = text.replace('Ó', 'o')
  text = text.replace('Ú', 'u')

# saca todos los doble espacios (siempre)
text = text.replace('  ', ' ')

print('\n -- Tamaño total del texto luego de la limpieza: {} caracteres'.format(len(text)))
print("\n -- Ejemplo luego de la limpieza: \n", text[:250])


 -- Tamaño total del texto luego de la limpieza: 18848 caracteres

 -- Ejemplo luego de la limpieza: 
  song cantata del adelantado don rodrigo diaz de carreras    introduccion   mastropiero era un apasionado de la investigacion historica   se pasaba largas horas en la biblioteca de la opulenta marquesa de quintanilla cuyos volumenes le apasionaban   


In [9]:
#@title Preparar texto 
tipo_datos = "palabras" #@param ["caracteres", "palabras"]

if tipo_datos == "caracteres":
  # The unique characters in the file
  auxText = text
  caracterJoin = ''
  vocab = sorted(set(auxText))
  print('{} caracteres distintos detectados'.format(len(vocab)))
else:
  auxText = text.split(' ')
  caracterJoin = ' '
  vocab = sorted(set(auxText))
  print('{} palabras distintas detectadas'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in auxText])

print('\nEjemplos de Codificación \n{')
for char,_ in zip(char2idx, range(10)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# Muestra ejemplo de cómo se mapean los caracteres a valores numéricos
print ('\n{} <-------- > {}'.format(repr(text[:13]), text_as_int[:13]))


1166 palabras distintas detectadas

Ejemplos de Codificación 
{
  ''  :   0,
  '1491':   1,
  'a' :   2,
  'aaahhh':   3,
  'abandonado':   4,
  'abandonar':   5,
  'abiertos':   6,
  'abominamos':   7,
  'aborigen':   8,
  'abrazo':   9,
  ...
}

' song cantata' <-------- > [  0 996 183 306  23 369 930 339 292 191   0   0   0]


In [10]:
#@title Armar secuencias de texto y formatear

# determinar el largo máximo de la secuencia
if ((len(text)//101)<1000):
  seq_length = 50
else:
  seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print("Largo de secuencias: ", seq_length)
print("\n")
print("Ejemplos por época: ", examples_per_epoch)

# Dividir en datos de entrenamiento y prueba, para ello divide el texto en secuencias donde 
#- la secuencia de la posición 0 a [seq_length] se considera de entrada, y 
#- la secuencia de la posición  [seq_length+1] al final es la de salida

# genera un vector de caracteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# procesa para generar las secuencias el largo deseado
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# muestra ejemplo
for item in sequences.take(5):
  print(repr(caracterJoin.join(idx2char[item.numpy()])))

# genera las secuencias de entrada y salida
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

datasetSeq = sequences.map(split_input_target)

print("\nDatasetSeq: ", datasetSeq, "\n")

# muestra ejemplo
for input_example, target_example in  datasetSeq.take(2):
  print ('Texto de Entrada: ', repr(caracterJoin.join(idx2char[input_example.numpy()])))
  print ('Texto  de Salida:', repr(caracterJoin.join(idx2char[target_example.numpy()])))
  print("\n")

Largo de secuencias:  50


Ejemplos por época:  369
' song cantata del adelantado don rodrigo diaz de carreras    introduccion   mastropiero era un apasionado de la investigacion historica   se pasaba largas horas en la biblioteca de la opulenta marquesa de quintanilla cuyos volumenes le apasionaban   alli supo mastropiero precisamente alli en'
'la biblioteca  de la existencia de un enigmatico personaje del siglo xv  el adelantado don rodrigo diaz de carreras  hijo de juana diaz y domingo de carreras   al principio de su investigacion  mastropiero supuso que don rodrigo pertenecia a la misma familia diaz  que'
'las celebres cortesanas angustias y dolores diaz  pero luego cotejando ciertas fechas  comprobo que angustias y dolores no provenian de esos diaz   mastropiero estaba por abandonar la investigacion  cuando encontro en la biblioteca de la marquesa  el viejo manuscrito de un anonimo poema epico '
'redactado sobre la base del diario de viaje  del adelantado don rodrigo diaz de carr

In [11]:
#@title Ejemplos

# muestra entrada y salida por cada caracter
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  Entrada: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  Salida Esperada: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  Entrada: 636 ('la')
  Salida Esperada: 138 ('biblioteca')
Step    1
  Entrada: 138 ('biblioteca')
  Salida Esperada: 0 ('')
Step    2
  Entrada: 0 ('')
  Salida Esperada: 292 ('de')
Step    3
  Entrada: 292 ('de')
  Salida Esperada: 636 ('la')
Step    4
  Entrada: 636 ('la')
  Salida Esperada: 459 ('existencia')


4) Especificar y preparar el modelo de la RNN a usar:

In [12]:
#@title Establecer modelo

# Seleccione el modelo a usar
capa_oculta_tipo = 'GRU'  #@param ["LSTM", "GRU", "RNN"]


# genera 'batch' de secuencias que se van a procesar en el entrenamiento

# Batch size
if capa_oculta_tipo == 'RNN':
  BATCH_SIZE = 1
else:
  BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 100000

dataset = datasetSeq.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print("Dataset: ", dataset, "\n")

# cantidad de neuronas RNN
rnn_units = 1024 

# The embedding dimension
embedding_dim = 256

# crea el modelo
model = RNNCustomModel(
    capa_oculta_tipo = capa_oculta_tipo,
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

# prepara variables auxiliares para el entrenamiento  de la RNN
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

# compila el modelo para el entrenamiento  de la RNN
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Forma vector predicción: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

print("\nModelo generado:")

model.summary()

Dataset:  <BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)> 

(64, 50, 1166) # (batch_size, sequence_length, vocab_size)
Forma vector predicción:  (64, 50, 1166)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.059937

Modelo generado:
Model: "GeneradorTexto"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
entrada (Embedding)          multiple                  298496    
_________________________________________________________________
oculta (GRU)                 multiple                  3938304   
_________________________________________________________________
salida (Dense)               multiple                  1195150   
Total params: 5,431,950
Trainable params: 5,431,950
Non-trainable params: 0
_________________________________________________________________


5) Entrenar la RNN:

In [22]:
#@title Entrenar

cant_epocas_entrenamiento =  1500#@param {type:"integer"}

# ejecutar el entrenamiento
# se recomientda usar GPU

history = model.fit(dataset, 
                    epochs=cant_epocas_entrenamiento)


Epoch 1/1500
1/1 [==============================] - 0s 124ms/step - loss: 3.4784
Epoch 2/1500
1/1 [==============================] - 0s 118ms/step - loss: 3.4383
Epoch 3/1500
1/1 [==============================] - 0s 110ms/step - loss: 3.3836
Epoch 4/1500
1/1 [==============================] - 0s 103ms/step - loss: 3.4104
Epoch 5/1500
1/1 [==============================] - 0s 101ms/step - loss: 3.4096
Epoch 6/1500
1/1 [==============================] - 0s 101ms/step - loss: 3.2743
Epoch 7/1500
1/1 [==============================] - 0s 104ms/step - loss: 3.2569
Epoch 8/1500
1/1 [==============================] - 0s 99ms/step - loss: 3.2522
Epoch 9/1500
1/1 [==============================] - 0s 93ms/step - loss: 3.1800
Epoch 10/1500
1/1 [==============================] - 0s 103ms/step - loss: 3.1344
Epoch 11/1500
1/1 [==============================] - 0s 94ms/step - loss: 3.1737
Epoch 12/1500
1/1 [==============================] - 0s 89ms/step - loss: 3.0500
Epoch 13/1500
1/1 [==========

6) Probar la RNN entrenada:

In [23]:
#@title Prepara el Generar de Texto y permite Grabar / Cargar el modelo entrenado

accion = "Grabar Modelo" #@param ["-", "Grabar Modelo", "Cargar Modelo"]
path_modelo = "/CantataGRU" #@param {type:"string"}

dirModelo = path + path_modelo

# instancia el modelo
if accion == "Cargar Modelo":
  # carga uno grabado
  genTexto = GeneradorTexto().cargar(dirModelo)
else:
  # genera uno nuevo en base al modelo entrenado
  genTexto = GeneradorTexto(model, char2idx, idx2char, caracterJoin)
  if accion == "Grabar Modelo":
    # lo graba al nuevo
    genTexto.grabar(dirModelo)

# ejecuta el modelo usando como entrada texto_inicial
print("\n\n--------------------------------------------------------------------------------------------\n")
print(genTexto.generar())
print("\n--------------------------------------------------------------------------------------------\n\n")




Pesos del modelo grabados en  gdrive/My Drive/IA/demoML/texto//CantataGRU
Datos asociados al modelo grabados engdrive/My Drive/IA/demoML/texto//CantataGRU/configModelo.csv




--------------------------------------------------------------------------------------------

                                                                                                    

--------------------------------------------------------------------------------------------




In [26]:
#@title Probar generación de texto 1

# Grado de "temperatura" u originalidad que va a generar el algoritmo:
# - cuanto más alto el valor, se genera texto "más sorprendente".
# - cuanto más bajo, se genera texto "más esperado".
originalidad =  1 #@param {type:"number"}

# Texto inicial para generar
texto_inicial = 'de aqui' #@param {type:"string" }

# Largo del texto a generar
largo_texto = 10 #@param {type:"integer" }

# ejecuta el modelo usando como entrada texto_inicial
print("\n\n--------------------------------------------------------------------------------------------\n")
print(genTexto.generar(originalidad, texto_inicial, largo_texto))
print("\n--------------------------------------------------------------------------------------------\n\n")



--------------------------------------------------------------------------------------------

de aquino hay en la biblioteca  es de una una

--------------------------------------------------------------------------------------------




In [27]:
#@title Probar generación de texto 2

# Grado de "temperatura" u originalidad que va a generar el algoritmo:
# - cuanto más alto el valor, se genera texto "más sorprendente".
# - cuanto más bajo, se genera texto "más esperado".
originalidad =  1 #@param {type:"number"}

# Texto inicial para generar
texto_inicial = 'el adelantado' #@param {type:"string" }

# Largo del texto a generar
largo_texto = 10 #@param {type:"integer" }

# ejecuta el modelo usando como entrada texto_inicial
print("\n\n--------------------------------------------------------------------------------------------\n")
print(genTexto.generar(originalidad, texto_inicial, largo_texto))
print("\n--------------------------------------------------------------------------------------------\n\n")



--------------------------------------------------------------------------------------------

el adelantadodon rodrigo diaz de una una una una una una

--------------------------------------------------------------------------------------------




In [28]:
#@title Probar generación de texto 3

# Grado de "temperatura" u originalidad que va a generar el algoritmo:
# - cuanto más alto el valor, se genera texto "más sorprendente".
# - cuanto más bajo, se genera texto "más esperado".
originalidad =  1 #@param {type:"number"}

# Texto inicial para generar
texto_inicial = 'dsdasd' #@param {type:"string" }

# Largo del texto a generar
largo_texto = 10 #@param {type:"integer" }

# ejecuta el modelo usando como entrada texto_inicial
print("\n\n--------------------------------------------------------------------------------------------\n")
print(genTexto.generar(originalidad, texto_inicial, largo_texto))
print("\n--------------------------------------------------------------------------------------------\n\n")



--------------------------------------------------------------------------------------------



KeyError: ignored